## <font color='darkblue'>Preface</font>
([article source](https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/)) <font size='3ptx'>**A simple and powerful regularization technique for neural networks and deep learning models is [dropout](https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/).**</font>

**In this post you will discover the dropout regularization technique and how to apply it to your models in Python with Keras.**

After reading this post you will know:
* How the dropout regularization technique works.
* How to use dropout on your input layers.
* How to use dropout on your hidden layers.
* How to tune the dropout level on your problem.

### <font color='darkgreen'>Agenda</font>
* <font size='3ptx'>[**Dropout Regularization For Neural Networks**](#sect1)</font>
* <font size='3ptx'>[**Dropout Regularization in Keras**](#sect2)</font>
* <font size='3ptx'>[**Using Dropout on the Visible Layer**](#sect3)</font>
* <font size='3ptx'>[**Using Dropout on Hidden Layers**](#sect4)</font>
* <font size='3ptx'>[**Tips For Using Dropout**](#sect5)</font>

<a id='sect1'></a>
## <font color='darkblue'>Dropout Regularization For Neural Networks</font>
**Dropout is a regularization technique for neural network models proposed by Srivastava, et al. in their 2014 paper [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](http://jmlr.org/papers/v15/srivastava14a.html) ([download the PDF](http://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)).**

**Dropout is a technique where randomly selected neurons are ignored during training**. They are “dropped-out” randomly. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass.

As a neural network learns, neuron weights settle into their context within the network. Weights of neurons are tuned for specific features providing some specialization. **Neighboring neurons become to rely on this specialization, which if taken too far can result in a fragile model too specialized to the training data. This reliant on context for a neuron during training is referred to complex co-adaptations.**

You can imagine that if neurons are randomly dropped out of the network during training, that other neurons will have to step in and handle the representation required to make predictions for the missing neurons. This is believed to result in multiple independent internal representations being learned by the network.

The effect is that **the network becomes less sensitive to the specific weights of neurons. This in turn results in a network that is capable of better generalization and is less likely to overfit the training data.**

In [13]:
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

<a id='sect2'></a>
## <font color='darkblue'>Dropout Regularization in Keras</font>
**Dropout is easily implemented by randomly selecting nodes to be dropped-out with a given probability (e.g. 20%) each weight update cycle. This is how Dropout is implemented in Keras. Dropout is only used during the training of a model and is not used when evaluating the skill of the model.**

Next we will explore a few different ways of using Dropout in Keras.

### <font color='darkgreen'>Data Set</font>
The examples will use the [**Sonar dataset**](https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)). This is a binary classification problem where the objective is to correctly identify rocks and mock-mines from sonar chirp returns. It is a good test dataset for neural networks because all of the input values are numerical and have the same scale.

The dataset can be [downloaded from the UCI Machine Learning repository](https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data). You can place the sonar dataset in your current working directory with the file name <font color='olive'>sonar.csv</font>. 

In [4]:
# Baseline Model on the Sonar Dataset
## load dataset
df = read_csv("../../datas/UCI/sonar.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


### <font color='darkgreen'>Baseline Model </font>
We will evaluate the developed models using scikit-learn with 10-fold cross validation, in order to better tease out differences in the results.

There are 60 input values and a single output value and the input values are standardized before being used in the network. The baseline neural network model has two hidden layers, the first with 60 units and the second with 30. Stochastic gradient descent is used to train the model with a relatively low learning rate and momentum.

The the full baseline model is listed below.

In [12]:
# split into input (X) and output (Y) variables
X = df.loc[:,0:59].astype(float)
Y = df.loc[:,60]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)
 
# baseline
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, activation='relu'))
    model.add(Dense(30,  activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    sgd = SGD(lr=0.01, momentum=0.8)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model
 
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 86.55% (6.65%)


Running the example generates an estimated classification accuracy of 86%.

<a id='sect3'></a>
## <font color='darkblue'>Using Dropout on the Visible Layer</font>
**Dropout can be applied to input neurons called the visible layer.**

In the example below we add a new [**Dropout layer**](https://keras.io/api/layers/regularization_layers/dropout/) between the input (<font color='brown'>or visible layer</font>) and the first hidden layer. The dropout rate is set to 20%, meaning one in 5 inputs will be randomly excluded from each update cycle.

Additionally, **as recommended in the original paper on Dropout, a constraint is imposed on the weights for each hidden layer, ensuring that the maximum norm of the weights does not exceed a value of 3**. This is done by setting the <font color='violet'>kernel_constraint</font> argument on the [**Dense class**](https://keras.io/api/layers/core_layers/dense/) when constructing the layers.

The **learning rate was lifted by one order of magnitude and the momentum was increase to 0.9**. These increases in the learning rate were also recommended in the original Dropout paper.

Continuing on from the baseline example above, the code below exercises the same network with input dropout.

In [16]:
# dropout in the input layer with weight constraint
def create_model_with_dropout():
    # create model
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(60,)))
    model.add(Dense(60, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    sgd = SGD(lr=0.1, momentum=0.9)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model
 
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model_with_dropout, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Visible: 86.02% (7.29%)


Running the example provides a small drop in classification accuracy (86.55% -> 86.02%), at least on a single test run.

<a id='sect4'></a>
## <font color='darkblue'>Using Dropout on Hidden Layers</font>
**Dropout can be applied to hidden neurons in the body of your network model.**

In the example below Dropout is applied between the two hidden layers and between the last hidden layer and the output layer. Again a dropout rate of 20% is used as is a weight constraint on those layers.

In [26]:
# dropout in hidden layers with weight constraint
def create_model_with_two_dropout():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    sgd = SGD(lr=0.15, momentum=0.90)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model
 
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model_with_two_dropout, epochs=300, batch_size=20, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Hidden: 84.21% (8.20%)


We can see that for this problem and for the chosen network configuration that using dropout in the hidden layers did not lift performance. In fact, performance was worse than the baseline (86.55% -> 84.17%). **It is possible that additional training epochs are required or that further tuning is required to the learning rate.**

<a id='sect5'></a>
## <font color='darkblue'>Tips For Using Dropout</font>
The original paper on Dropout provides experimental results on a suite of standard machine learning problems. As a result they provide a number of useful heuristics to consider when using dropout in practice.
* Generally, use a small dropout value of 20%-50% of neurons with 20% providing a good starting point. A probability too low has minimal effect and a value too high results in under-learning by the network.
* Use a larger network. You are likely to get better performance when dropout is used on a larger network, giving the model more of an opportunity to learn independent representations.
* Use dropout on incoming (visible) as well as hidden units. Application of dropout at each layer of the network has shown good results.
* Use a large learning rate with decay and a large momentum. Increase your learning rate by a factor of 10 to 100 and use a high momentum value of 0.9 or 0.99.
* Constrain the size of network weights. A large learning rate can result in very large network weights. Imposing a constraint on the size of network weights such as max-norm regularization with a size of 4 or 5 has been shown to improve results.

## <font color='darkblue'>More Resources on Dropout</font>
Below are some resources that you can use to learn more about dropout in neural network and deep learning models.
* [Dropout: A Simple Way to Prevent Neural Networks from Overfitting (original paper)](http://jmlr.org/papers/v15/srivastava14a.html).
* [Improving neural networks by preventing co-adaptation of feature detectors.](https://arxiv.org/abs/1207.0580)
* [How does the dropout method work in deep learning? on Quora.](https://www.quora.com/How-does-the-dropout-method-work-in-deep-learning)